## Parse out main text

In [5]:
import os
import sys 
from lxml import etree
from unidecode import unidecode

In [6]:
# Update env and pkg paths.

sys.path.extend(['/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages'])
print(sys.path)

['/data/xzuo/Projects/dissertation/dataset_URL_parser/paper_oriented', '/usr/local/anaconda3/lib/python39.zip', '/usr/local/anaconda3/lib/python3.9', '/usr/local/anaconda3/lib/python3.9/lib-dynload', '', '/home/xzuo/.local/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages', '/usr/local/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/usr/local/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/xzuo/.ipython', '/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages', '/data/xzuo/.conda/envs/dissertation/lib/python38.zip', '/data/xzuo/.conda/envs/dissertation/lib/python3.8', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/lib-dynload', '/data/xzuo/.conda/envs/dissertation/lib/python3.8/site-packages']


In [7]:
import pubmed_parser as pp
import pandas as pd
import re

In [8]:
# Get pmid from LitCovid.

id_list_path = "./litcovid_pmid/litcovid_16052022.tsv"
csv_directory_path = "./litcovid_pmcid_lists"
download_directory_path ="./litcovid_pmc_fulltext/xml"

current_filelist = os.listdir(download_directory_path)

In [ ]:
# Remove inaccessible files.

inaccessible_articles = []

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    try:
        pmc_tree = etree.parse(pmc_xml_path)
    except:
        # print("Remove file %s." % xml_name)
        inaccessible_articles.append(xml_name)
        current_filelist.remove(xml_name)

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    pmc_tree = etree.parse(pmc_xml_path)
    meta = pmc_tree.find(".//article-meta")
    if meta is None:
        inaccessible_articles.append(xml_name)
        current_filelist.remove(xml_name)
#         print("File %s is not accessible." % xml_name)
    else:
        continue

print("There are %d articles not accessible." % len(inaccessible_articles))

for i, xml_name in enumerate(inaccessible_articles):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    if os.path.exists(pmc_xml_path):
        os.remove(pmc_xml_path)
    else:
        print("Error: File %s does not exist." % xml_name)

In [8]:
# Parse out text.
articles = []

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    pmc_tree = etree.parse(pmc_xml_path)
    meta = pmc_tree.find(".//article-meta")
    try:
        metadata = pp.parse_pubmed_xml(pmc_xml_path)
        full_text = pp.parse_pubmed_paragraph(pmc_xml_path, all_paragraph=True)
        paras = []
        for para in full_text:
            paras.append(para['text'].strip())
            main_text = "\n".join(paras)
        pmc_article = {"pmid": metadata['pmid'], "pmcid": metadata['pmc'], 
                       "title": metadata['full_title'], "abstract": metadata['abstract'], 
                       "main text": main_text}
        articles.append(pmc_article)
    except:
        os.remove(pmc_xml_path)
        print("File %s is not accessible. Removed." % xml_name)

df = pd.DataFrame(articles)
display(df)

,pmid,pmcid,title,abstract,main text
0,34776649,8572530,Subjektives Befinden und Arbeitsfähigkeit nach...,Hintergrund Seit dem 29.01.2021 wurde der C...,Gegen SARS-CoV‑2 wurden Impfstoffe mit völlig ...
1,34098882,8182346,"Web-based, rapid and contactless management of...",Background During the SARS-CoV-2 pandemic a...,"From the end of February 2020 onwards, many pa..."
2,34541298,8441688,Molecular Level Dissection of Critical Spike M...,Abstract SARS‐CoV‐2 virus during its spread ...,Severe Acute Respiratory Syndrome Corona Virus...
3,32302815,7151272,Psychological status of surgical staff during ...,Highlight • The mental health problems o...,"Dear editor,\nIn the outbreak of novel coronav..."
4,34295171,8291382,The Immunogenicity of Hyaluronic Fillers and I...,Abstract Hyaluronic acid (HA) is a glycosami...,"Hyaluronic acid (HA) is a glycosaminoglycan, a..."
...,...,...,...,...,...
61889,34115405,8420322,The presence of SARS‐CoV‐2 virus in semen samp...,Abstract The SARS‐CoV‐2 set off a pandemic i...,Coronavirus (SARS‐CoV‐2) induced a pandemic al...
61890,35029196,8757937,Does influenza vaccination help reduce inciden...,Abstract To facilitate the understanding of ...,Corona virus disease 2019 (COVID-19) was decla...
61891,35414326,9115793,Myocarditis following COVID-19 vaccination: in...,ABSTRACT Introduction Vaccines have demons...,The concept of introducing substrate resemblin...
61892,34655522,8514195,Safety and immunogenicity of CpG 1018 and alum...,"Background MVC-COV1901, a recombinant prote...",\nEvidence before this study\nTo understand th...


## General Search

In [9]:
import pysbd
import unidecode
from nltk.tokenize import word_tokenize

### Search dataset description sentences

In [9]:
seg = pysbd.Segmenter(language="en", clean=False)
pattern = re.compile(r'((?:train|test|validation|testing|trainings?)\s*(?:set|data))|(benchmarks?)|(data\s*(?:set|base)s?)|(corpus)|(corpora)|(tree\s*bank)|(collections?)')

for i, article in enumerate(articles):
    all_text = article['title'] + ".\n" + article['abstract'] + "\n" + article['main text']
    article['sentences'] = seg.segment(all_text)
    article['has_pattern'] = []
    for j, sent in enumerate(article['sentences']):
        if pattern.search(sent):
            article['has_pattern'].append(sent)
    if i != 0 and i % 1000 == 0:
        print("Searched %d full-text articles." % i)
        break

Searched 1000 full-text articles.


In [10]:
qualified_articles = []
for article in articles:
    if "has_pattern" in article and len(article['has_pattern']) > 20:
        qualified_articles.append(article)

qualified_df = pd.DataFrame(qualified_articles)
display(qualified_df)

,pmid,pmcid,title,abstract,main text,sentences,has_pattern
0,34847386,8609674,Uncertainty-aware convolutional neural network...,Deep learning (DL) has shown great success in...,Novel Corona Virus disease 2019 (COVID-19) has...,[Uncertainty-aware convolutional neural networ...,[The proposed method is evaluated on the three...
1,34976572,8675556,Detecting SARS-CoV-2 From Chest X-Ray Using Ar...,Chest radiographs (X-rays) combined with Deep...,The Severe Acute Respiratory Syndrome Coronavi...,[Detecting SARS-CoV-2 From Chest X-Ray Using A...,"[However, questions remain regarding the accur..."
2,34352699,8249716,Conventional and microfluidic methods for airb...,Graphical abstract With the COVID-19 pan...,"Throughout history, infectious diseases have p...",[Conventional and microfluidic methods for air...,[Creating targeted and effective interventions...
3,32836689,7236753,Competitive pricing of substitute products und...,There has been an increased interest in optim...,Retailers frequently rely on suppliers that ar...,[Competitive pricing of substitute products un...,[Section 3 is devoted to benchmark modeling of...
4,34070422,8226892,Shall I Work with Them? A Knowledge Graph-Base...,We consider the prediction of future research...,The development of knowledge graph-based appro...,"[Shall I Work with Them? , A Knowledge Graph-B...",[We benchmark the proposed approach against cl...
5,34139910,8216038,Protecting Privacy and Transforming COVID-19 C...,Objectives Federal open-data initiatives th...,Federal open-data initiatives that promote inc...,[Protecting Privacy and Transforming COVID-19 ...,[We describe how CDC designed and produces 2 d...
6,33935584,8068562,A Novel Ensemble-based Classifier for Detectin...,"The recently discovered coronavirus, SARS-CoV...","The SARS-CoV-2 that causes COVID-19, colloquia...",[A Novel Ensemble-based Classifier for Detecti...,"[In this paper, a synthetic dataset of COVID-1..."
7,33728416,7953447,VERSO: A comprehensive framework for the infer...,"Summary We introduce VERSO, a two-step frame...",The outbreak of coronavirus disease 2019 (COVI...,[VERSO: A comprehensive framework for the infe...,[The application to large-scale datasets demon...
8,35071990,8767797,Development and implementation of a COVID-19 c...,Introduction Convalescent Plasma therapy is...,"In late 2019, patients presenting an atypical ...",[Development and implementation of a COVID-19 ...,[Methods A multicentric convalescent plasma c...
9,32898686,7474840,"Digital technology, tele-medicine and artifici...",The simultaneous maturation of multiple digit...,2020 marked the synchronous maturation of seve...,"[Digital technology, tele-medicine and artific...",[A review of different tele-health models and ...


### Search dataset description sentences

In [21]:
seg = pysbd.Segmenter(language="en", clean=False)
pattern = re.compile(r'(N3C)|(National COVID Cohort Collaborative)|(GenBank)|\
                     (GISAID)|(Nextstrain)|(OpenICPSR)|(TCIA)|(GEO)|(Gene Expression Omnibus)\
                     (CORD-19)|(LitCovid)|(NCBI Virus)')

for i, article in enumerate(articles):
    # all_text = article['title'] + ".\n" + article['abstract'] + "\n" + article['main text']
    title_abstract_text = article['title'] + ".\n" + article['abstract']
    article['sentences'] = seg.segment(title_abstract_text)
    # article['sentences'] = seg.segment(all_text)
    article['has_pattern'] = []
    for j, sent in enumerate(article['sentences']):
        if pattern.search(sent):
            article['has_pattern'].append(sent)
    if i != 0 and i % 1000 == 0:
        print("Searched %d full-text articles." % i)
        # break

print("Search Completed.")

Searched 1000 full-text articles.
Searched 2000 full-text articles.
Searched 3000 full-text articles.
Searched 4000 full-text articles.
Searched 5000 full-text articles.
Searched 6000 full-text articles.
Searched 7000 full-text articles.
Searched 8000 full-text articles.
Searched 9000 full-text articles.
Searched 10000 full-text articles.
Searched 11000 full-text articles.
Searched 12000 full-text articles.
Searched 13000 full-text articles.
Searched 14000 full-text articles.
Searched 15000 full-text articles.
Searched 16000 full-text articles.
Searched 17000 full-text articles.
Searched 18000 full-text articles.
Searched 19000 full-text articles.
Searched 20000 full-text articles.
Searched 21000 full-text articles.
Searched 22000 full-text articles.
Searched 23000 full-text articles.
Searched 24000 full-text articles.
Searched 25000 full-text articles.
Searched 26000 full-text articles.
Searched 27000 full-text articles.
Searched 28000 full-text articles.
Searched 29000 full-text arti

In [22]:
qualified_articles = []
for article in articles:
    if "has_pattern" in article and len(article['has_pattern']) > 0:
        qualified_articles.append(article)

qualified_df = pd.DataFrame(qualified_articles)
display(qualified_df)

,pmid,pmcid,title,abstract,main text,sentences,has_pattern
0,34765026,8576622,Testicular injury during SARS-CoV-2 infection ...,"Coronavirus disease 2019 (COVID-19), caused b...","Coronavirus disease 2019 (COVID-19), caused by...",[Testicular injury during SARS-CoV-2 infection...,"[For this, the ACE2-expressing cell compositio..."
1,35422859,9002903,Combination of Enrichment Using Gene Ontology ...,Introduction The severity of coronavirus di...,A coronavirus is a group of viruses from the s...,[Combination of Enrichment Using Gene Ontology...,[We collected the transcriptomics data from GE...
2,33166392,7778958,LitCovid: an open database of COVID-19 literature,Abstract Since the outbreak of the current p...,"Since the outbreak of the COVID-19 pandemic, r...",[LitCovid: an open database of COVID-19 litera...,[LitCovid: an open database of COVID-19 litera...
3,34821594,8625834,Characterization and Pathogenic Speculation of...,Patients with coronavirus disease 2019 (COVID...,Infection with severe acute respiratory syndro...,[Characterization and Pathogenic Speculation o...,[Scientific articles were retrieved by searchi...
4,32022275,7166327,The first two cases of 2019‐nCoV in Italy: Whe...,"Abstract A novel Coronavirus , 2019‐nCoV, ha...","An ongoing epidemic by a new Coronavirus, name...",[The first two cases of 2019‐nCoV in Italy: Wh...,[A maximum clade credibility tree has been bui...
...,...,...,...,...,...,...,...
69,35005058,8734487,Evaluation of multiple open-source deep learni...,Abstract. Purpose : Chest x-rays are complex...,The early radiological features of viral pneum...,[Evaluation of multiple open-source deep learn...,[Approach : We tested three open-source deep l...
70,34675627,8520483,Construction and Investigation of Competing En...,Introduction The current COVID-19 pandemic ...,The Coronaviruses (CoVs) are a varied group of...,[Construction and Investigation of Competing E...,[Methods The RNA sequencing data of SARS-CoV-...
71,32773643,7493783,Genomic variance of Open Reading Frames (ORFs)...,Background: The outbreak of severe acute res...,The outbreak of severe acute respiratory syndr...,[Genomic variance of Open Reading Frames (ORFs...,[Methods: We apply genomic alignment analysis...
72,34019643,8218201,eVITTA: a web-based visualization and inferenc...,Abstract Transcriptome profiling is essentia...,Transcriptome profiling is an essential techni...,[eVITTA: a web-based visualization and inferen...,"[Additionally, there is no systematic way to e..."


In [6]:
txt_directory = "./fulltext_txt_nih/"

In [9]:
# parser = CoreNLPParser(url='http://localhost:9000')

for article in article_batch:
    # print(article['pmcid'])
    all_text = article['title'] + ".\n" + article['abstract'] + "\n" + article['main text'].strip()
    # Format lines.
    raw_lines = all_text.splitlines()
    # lines = [line.strip() for line in raw_lines if len(word_tokenize(line)) > 10]
    lines = []
    for line in raw_lines:
        line_decode = unidecode.unidecode(line)
        if len(word_tokenize(line_decode)) > 10:
            lines.append(line.strip())
        else:
            continue
    new_lines = []
    for line in lines:
        if not new_lines or not(line.startswith((")", "-", ","))):
            new_lines.append(line)
        else:
            new_lines[-1] += line
    new_text_per_article = "\n\n".join(new_lines)
    txt_name = "PMC{}.txt".format(article['pmcid'])
    output_path = os.path.join(txt_directory, txt_name)
    with open(output_path, "w", encoding='utf-8') as output_file:
        output_file.write(new_text_per_article)

TypeError: string indices must be integers

## Detailed search

In [5]:
from nltk.tokenize import sent_tokenize

In [19]:
pmcs_has_pattern = top20_df['pmcid'].to_list()
pmcpath_has_pattern = ["PMC" + pmcid + ".xml" for pmcid in pmcs_has_pattern]
print(pmcpath_has_pattern[:10])

['PMC8675677.xml', 'PMC7989622.xml', 'PMC8180385.xml', 'PMC8336924.xml', 'PMC8653328.xml', 'PMC8362655.xml', 'PMC8216038.xml', 'PMC8319175.xml', 'PMC7823180.xml', 'PMC8468495.xml']


In [31]:
count = 0
count_all = 0
dataset_sents = []

for xml_name in pmcpath_has_pattern[:10]:
    xml_path = os.path.join(download_directory_path, xml_name)
    metadata = pp.parse_pubmed_xml(xml_path)
    full_text = pp.parse_pubmed_paragraph(xml_path, all_paragraph=True)
    for para in full_text:
        sents = sent_tokenize(para['text'])
        for sent in sents:
            count_all += 1
            if re.search(pattern, sent):
                count += 1
                sentence = {"pmid": metadata['pmid'], "pmcid": metadata['pmc'], "sentence": sent}
                dataset_sents.append(sentence)

In [34]:
sent_df = pd.DataFrame(dataset_sents)
display(sent_df)
sent_df.to_csv('./top10_sents.tsv', sep='\t', index=False)  

,pmid,pmcid,sentence
0,34914688,8675677,Using this graphical model called the Stochast...
1,34914688,8675677,"To this end, we first analyzed a large dataset..."
2,34914688,8675677,We then analyzed other datasets to determine w...
3,34914688,8675677,"Early in the pandemic, we implemented the Stoc..."
4,34914688,8675677,We chose to first perform the same analysis ov...
...,...,...,...
851,34573790,8468495,"For our analysis, we used the synthetic and re..."
852,34573790,8468495,Additional synthetic and real datasets exist b...
853,34573790,8468495,The synthetic datasets for the three different...
854,34573790,8468495,The synthetic versions of some datasets and th...


## Parse full-text with section headers

In [14]:
# Generate PMC article file names.

# pmc_to_extract = df_rand_100['pmcid'].to_list()
pmc_to_extract = remain_articles
pmc_to_extract_filenames = ["PMC" + pmcid + ".xml" for pmcid in pmc_to_extract]
print(pmc_to_extract_filenames[:10])

['PMC8475511.xml', 'PMC8609674.xml', 'PMC8675556.xml', 'PMC7236753.xml', 'PMC8216038.xml', 'PMC8767797.xml', 'PMC7474840.xml', 'PMC7799233.xml', 'PMC7543791.xml', 'PMC8312471.xml']


In [17]:
txt_directory = "./fulltext_txt/"

In [24]:
article_batch = ["PMC8491001.xml", "PMC9169748.xml", "PMC7970480.xml", "PMC8662089.xml"]

In [25]:
# Generate text files for annotation.

for i, xml_name in enumerate(article_batch):
    xml_path = os.path.join(download_directory_path, xml_name)
    metadata = pp.parse_pubmed_xml(xml_path)
    full_text = pp.parse_pubmed_paragraph(xml_path, all_paragraph=True)
    
    paras = []
    for para in full_text:
        paras.append(para['text'].strip())
        main_text = "\n".join(paras)
        # paras.append(paragraph['all text'])
    all_text_per_article = metadata['full_title'] + ".\n" + metadata['abstract'] + "\n" + main_text
    
    # Format lines.
    lines = all_text_per_article.splitlines()
    lines = [line for line in lines if len(line) > 2]
    new_lines = []
    for line in lines:
        if not new_lines or not(line.startswith((")", "-", ","))):
            new_lines.append(line)
        else:
            new_lines[-1] += line
    new_text_per_article = "\n\n".join(new_lines)
    # print(all_text_per_article)
    
    txt_name = xml_name.replace(".xml", ".txt")
    output_path = os.path.join(txt_directory, txt_name)
    with open(output_path, "w", encoding='utf-8') as output_file:
        output_file.write(new_text_per_article)

## Parse out Mesh and Methods

In [34]:
section_pattern = re.compile(r"\bmethods?\b|\bMethodolog[yi]\b", re.IGNORECASE)

In [35]:
# rwd_sec_query = re.compile(r'\b(CMS|Medicare|Medicaid|SEER|Optum|Cinformatics|Data Mart|MIMIC|NHANES|National Health Survey|Nutrition Examination Survey|NAMCS|Ambulatory Medical Care Survey|NIS|Inpatient Sample|VHA|CDW|Veterans Health Administration|OptumLabs|Data Warehouse|N3C|National COVID Cohort Collaborative)\b', re.IGNORECASE)
# omics_sec_query = re.compile(r'(sequence read archive|SRA)|covid-19 host genetics initiative|covid-19 cell atlas|(european nucleotide archive|ENA)|broad terra|2019nCoVR|(covid-19 genome sequence dataset)|(european bioinformatics institute|embl-ebi)|genbank|gaid|gisa(\\w+)|ncbi virus|nextstrain|reactome')
rwd_sec_query = re.compile(r'\b(Optum|Cinformatics|Data Mart|MIMIC|National Health Survey|Nutrition Examination Survey|Ambulatory Medical Care Survey|Corporate Data Warehouse|N3C|National COVID Cohort Collaborative)\b')
omics_sec_query = re.compile(r'(Sequence Read Archive|SRA)|(Host Genetics Initiative|HGI)|COVID-19 Cell Atlas|(European Nucleotide Archive|ENA)|broad terra|2019nCoVR|(COVID-19 Data Portal)|[Gg]en[Bb]ank|(GISAID)|(Gene Expression Omnibus|GEO)|(\b[Nn]extstrain\b)|Reactome|(NCBI Virus)|(PubChem)|(Kyoto Encyclopedia of Genes and Genomes|KEGG))')

In [36]:
# Parse out text.
articles = []

for i, xml_name in enumerate(current_filelist):
    pmc_xml_path = os.path.join(download_directory_path, xml_name)
    pmc_tree = etree.parse(pmc_xml_path)
    meta = pmc_tree.find(".//article-meta")
    metadata = pp.parse_pubmed_xml(pmc_xml_path)
    full_text = pp.parse_pubmed_paragraph(pmc_xml_path, all_paragraph=True)
    paras = []
    for j, para in enumerate(full_text):
        if section_pattern.search(para['section']):
            paras.append(para['text'])
        else:
            continue
    main_text = "\n".join(paras)
    pmc_article = {"pmid": metadata['pmid'], "pmcid": metadata['pmc'], 
                       "title": metadata['full_title'], "abstract": metadata['abstract'], 
                       "main text": main_text}
    articles.append(pmc_article)
    

print("Found %d articles." % len(articles))

AttributeError: 'NoneType' object has no attribute 'find'

In [16]:
print(len(articles))

140649


In [17]:
matched_articles = []
for i, article in enumerate(articles):
    text = article['main text']
    rwd_match = rwd_sec_query.search(text)
    omics_match = omics_sec_query.search(text)
    if rwd_match:
        print("%s\t%s" % (article['pmcid'], rwd_match.group(0)))
        matched_articles.append(article)
    elif omics_match:
        print("%s\t%s" % (article['pmcid'], omics_match.group(0)))
    else:
        continue

8014456	SRA
8673727	GISAID
8598683	Medicare
7853258	GISAID
8847733	GISAID
8222831	Genbank
7405216	Nutrition Examination Survey
8156524	Reactome
8491001	VHA
9169748	GEO
7970480	Data Warehouse
8662089	ENA
7898962	Medicare
9088623	nextstrain
8689746	GISAID
7454936	GISAID
8520125	Medicaid
8235306	nextstrain
8876352	GISAID
8351078	Data Warehouse
7662030	GEO
8168855	Host Genetics Initiative
8065152	GISAID
8812349	Medicaid
7953981	GISAID
8420260	ENA
8096765	GISAID
8014828	GISAID
8795823	VHA
7675408	GISAID
7099638	GISAID
8600389	Medicare
8755559	GISAID
7450963	GISAID
8080116	Medicare
8250548	SRA
9021587	VHA
8522425	GISAID
8920877	GISAID
7745650	GISAID
7404845	Medicare
8299148	Medicare
8307577	GISAID
7653226	Data Warehouse
8014766	Medicare
8341697	GISAID
8919776	Medicaid
7825889	Gene Expression Omnibus
8830142	Medicare
7716286	Medicaid
7851715	GISAID
9056975	GISAID
8601222	GISAID
7313917	GISAID
7861616	nextstrain
8649449	Medicare
7335245	SRA
8888350	Nextstrain
7816100	Data Warehouse
8573839	GIS

8914131	ENA
8299215	Medicare
7251277	Nextstrain
8171993	SRA
7228393	GISAID
8713434	Genbank
8947833	Gene Expression Omnibus
8725927	GISAID
8855654	GISAID
8018669	SRA
7463154	VHA
7640577	NIS
9021583	Veterans Health Administration
8943844	GISAID
8447490	Sequence Read Archive
7231477	GISAID
7233242	ENA
8294486	Data Warehouse
8097105	VHA
7890338	Optum
8138329	CMS
8803346	Nextstrain
8076216	GISAID
8035651	GISAID
7480334	SRA
8485368	ENA
8184080	GISAID
7684019	Medicare
8739765	Medicare
8280612	GISAID
8901586	SRA
9088325	Data Warehouse
8762488	Medicare
8013796	Medicare
7684018	Veterans Health Administration
7485934	Medicare
8441748	ENA
8349402	nextstrain
8279096	CDW
8880348	NIS
7949745	GISAID
8479448	Medicare
8102052	Medicare
8809573	Medicare
7832795	GISAID
8224693	Medicare
8104036	Medicaid
7971693	GISAID
8908924	Nutrition Examination Survey
7251363	Medicare
8233546	Medicaid
8547852	GISAID
7437610	CMS
7436700	GISAID
8661962	GISAID
7917268	Data Warehouse
8895417	ENA


In [18]:
txt_directory="./methods_txt/"

In [25]:
for i, article in enumerate(matched_articles):
    xml_name = "PMC{}.xml".format(article['pmcid'])
    xml_path = os.path.join(download_directory_path, xml_name)
    metadata = pp.parse_pubmed_xml(xml_path)
#     try:
#         full_text = pp.parse_pubmed_paragraph(pmc_xml_path, all_paragraph=True)
#     except:
#         print("Error in %s." % xml_name)
#         os.remove(xml_path)
    full_text = pp.parse_pubmed_paragraph(pmc_xml_path, all_paragraph=True)
    paras = []
    for j, para in enumerate(full_text):
        if section_pattern.search(para['section']):
            paras.append(para['text'])
        else:
            continue
    main_text = "\n".join(paras)
    
    # Format lines.
    lines = all_text_per_article.splitlines()
    lines = [line for line in lines if len(line) > 2]
    new_lines = []
    for line in lines:
        if not new_lines or not(line.startswith((")", "-", ","))):
            new_lines.append(line)
        else:
            new_lines[-1] += line
    new_text_per_article = "\n\n".join(new_lines)
    # print(all_text_per_article)
    
    txt_name = xml_name.replace(".xml", ".txt")
    output_path = os.path.join(txt_directory, txt_name)
    with open(output_path, "w", encoding='utf-8') as output_file:
        output_file.write(new_text_per_article)

Error: it was not able to read a path, a file-like object, or a string as an XML


XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)